In [1]:
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import zipfile
import os


In [2]:
# Upload the zip file
from google.colab import files
uploaded = files.upload()

# Unzip the uploaded file
for filename in uploaded.keys():
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('/content/Chicken_Diseases')

# Set the base directory path
base_dir = '/content/Chicken_Diseases/Chicken_Diseases'
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')




Saving Chicken Diseases4.zip to Chicken Diseases4.zip


In [ ]:
# ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    batch_size=64,
                                                    class_mode='binary')

valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    target_size=(224, 224),
                                                    batch_size=64,
                                                    class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(224, 224),
                                                  batch_size=64,
                                                  class_mode='binary')


Found 200 images belonging to 2 classes.
Found 28 images belonging to 2 classes.
Found 86 images belonging to 2 classes.


In [ ]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
# Calculate steps per epoch based on batch size and number of samples
total_number_of_samples_in_training_set = train_generator.samples
steps_per_epoch = max(1, total_number_of_samples_in_training_set // train_generator.batch_size)

# Check if steps_per_epoch is greater than 0
if steps_per_epoch <= 0:
    raise ValueError("steps_per_epoch must be greater than 0")

# Train the model
history = model.fit(train_generator,
                    epochs=25,
                    validation_data=valid_generator,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=max(1, valid_generator.samples // valid_generator.batch_size))


Epoch 1/25
3/3 [==============================] - 3s 1s/step - loss: 0.7076 - accuracy: 0.5000 - val_loss: 0.6964 - val_accuracy: 0.5000
Epoch 2/25
3/3 [==============================] - 4s 1s/step - loss: 0.6995 - accuracy: 0.4926 - val_loss: 0.6875 - val_accuracy: 0.5000
Epoch 3/25
3/3 [==============================] - 2s 1s/step - loss: 0.6925 - accuracy: 0.4559 - val_loss: 0.6726 - val_accuracy: 0.5000
Epoch 4/25
3/3 [==============================] - 2s 570ms/step - loss: 0.6785 - accuracy: 0.4926 - val_loss: 0.6484 - val_accuracy: 0.5000
Epoch 5/25
3/3 [==============================] - 2s 577ms/step - loss: 0.6435 - accuracy: 0.5882 - val_loss: 0.6278 - val_accuracy: 0.7143
Epoch 6/25
3/3 [==============================] - 3s 751ms/step - loss: 0.6200 - accuracy: 0.7500 - val_loss: 0.6094 - val_accuracy: 0.7143
Epoch 7/25
3/3 [==============================] - 3s 914ms/step - loss: 0.6012 - accuracy: 0.7448 - val_loss: 0.6699 - val_accuracy: 0.7143
Epoch 8/25
3/3 [=============

In [ ]:
# Save the trained model as an .h5 file
model.save('chicken_disease_classifier.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from google.colab import files

# Download the model file to your local system
files.download('chicken_disease_classifier.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>